In [1]:
# Initialize these:
typeImg = "cartoon" # photo/ cartoon/ top"
numImg = "50" # 5 or 50
path = "E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads"  # (edit this )

In [2]:
# # Include imports
import glob
from skimage.transform import resize
from skimage.io import imread, imsave, imshow
import os
import pandas as pd
import numpy as np
import skimage.color as skcolor
import matplotlib.pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import time

In [3]:
colNames = ["col_"+str(i) for i in range(58)] 
allConcepts = ['mango','watermelon','honeydew','cantaloupe','grapefruit','strawberry','raspberry','blueberry','avocado','orange','lime','lemon']

# Read data
# Get the color ratings data -From file data.csv
data = pd.read_csv("data.csv",names = colNames)

# Get the LAB coordinates of colors from file Lab.csv
colorData = pd.read_csv("Lab.csv",header=None)
colorDataRGB = pd.read_csv("RGB.csv",header=None)

category = pd.read_csv("Category.csv",header=None)

# if not os.path.exists( path + "/trainResized" ):
#    os.makedirs( path + "/trainResized/" )


In [4]:
# Count total number of images
count = 0
for i in allConcepts:
    print(path + "/" +typeImg +"/" +i+"/*.*")
    trainFiles = glob.glob( path + "/" +typeImg +"/" +i+"/*.*" )
    count = count + len(trainFiles)
    #print(count)
print("Category: ",typeImg)
print("Total images : ",count)

E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/mango/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/watermelon/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/honeydew/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/cantaloupe/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/grapefruit/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/strawberry/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/raspberry/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/blueberry/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/avocado/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/orange/*.*
E:/GitRepo/Color-Concept-Associations-using-Google-Images/downloads/cartoon/lime/*.*
E:/GitRepo/Color-Conc

In [5]:
import matlab.engine
category = pd.read_csv("Category.csv",header=None)

def getCategoriesMatrix(imageFileName,sz):
    ## Get total pixel counts of all 11 categories for this imageFile
    eng = matlab.engine.start_matlab()
    imageCat = np.array(eng.getCategories(imageFileName,sz))
    eng.quit()
     # imageCat : sz x sz matrix where each pixel has its category
    return imageCat

def getSegmentedImage(imageFileName,sz): # Returns segmented pixels in LAB and its corresponding color category.
    eng = matlab.engine.start_matlab()
    segImg, segImgCat = eng.activeContourSegmentation(imageFileName,sz, nargout=2)
    eng.quit()
    segImg = np.array(segImg)/255;
    segImg = skcolor.rgb2lab(segImg)
    segImgCat = np.array(segImgCat)
    return segImg, segImgCat


In [ ]:
numFt = 186
X = np.zeros((count*58,numFt+1) )  # +1 for label #Number of features [] # Feature Vector -- 58N rows
init = time.time()
rowNum = -1
strtInd = []  # To keep track of the starting indices of fruits in the feature matrix
cvInd = 1 
counter = 0
sz = 100    # Size of image: 100 by 100


##
Rc = [1,10,20,30,40]   # Radii thresholds
Li = [1, 10, 20, 30, 40]
R = [1, 10, 20, 30, 40]     # same as L
hueAngle = [5, 10, 20, 30, 40] # Hue Angle
##
for conceptNum,concept in enumerate(allConcepts):
    t0 = time.time()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>\n",conceptNum+1,": Concept : ",concept)
    # Get label for corresponding fruit (for given color)
    #label = data.loc[conceptNum]['col_'+str(color)]
    ct = 0    # Keeps track of images inside a category
    strtInd.append(counter)
    trainFiles = glob.glob( path + "/" +typeImg +"/" +concept+"/*.*" )
    print("\tTotal number of images for ",concept," : ",len(trainFiles))
    t = []
    for m, nameFile in enumerate(trainFiles):
        t1 = time.time()
        #print("\nReading file ......")
        ct = ct + 1
        
        image = imread( nameFile )
        imageResized = resize( image, (sz,sz),anti_aliasing=True,mode='reflect')
        img = imageResized
        # Convert the pixels in  image to LAB space
        imgLAB = skcolor.rgb2lab(img)
        segmentedImg,segmentedImgCat = getSegmentedImage(nameFile,sz)
        imageWithCategories = getCategoriesMatrix(nameFile,sz)
        
        i1 = imgLAB[27:72,27:72,:] 
        i2 = imgLAB[19:83,19:83,:]
        i3 = imgLAB[12:90,12:90,:]
        i4 = imgLAB[6:96,6:96,:]
        i5 = np.copy(imgLAB)
        i6 = np.copy(segmentedImg)
        
        ii1 = imageWithCategories[27:72,27:72].flatten()
        ii2 = imageWithCategories[19:83,19:83].flatten()
        ii3 = imageWithCategories[12:90,12:90].flatten()
        ii4 = imageWithCategories[6:96,6:96].flatten()
        ii5 = np.copy(imageWithCategories).flatten()
        ii6 = np.copy(segmentedImgCat).flatten()
        
        categoryImg1 = [sum(1 for p in ii1 if p-1 == k)/ii1.size for k in range(11)]
        categoryImg2 = [sum(1 for p in ii2 if p-1 == k)/ii2.size for k in range(11)]
        categoryImg3 = [sum(1 for p in ii3 if p-1 == k)/ii3.size for k in range(11)]
        categoryImg4 = [sum(1 for p in ii4 if p-1 == k)/ii4.size for k in range(11)]
        categoryImg5 = [sum(1 for p in ii5 if p-1 == k)/ii5.size for k in range(11)]
        categoryImg6 = [sum(1 for p in ii6 if p-1 == k)/ii6.size for k in range(11)]
        
        
        for color in range(58):
            rowNum+=1
            counter += 1
            colCt = 0
            # Thus begin the long long process
            # Get spherical and add till 
            colorLAB = np.array(colorData.loc[color])   #get LAB values for specific color
            
            ##   Columns 1-30
            for i in range(6):
                if i == 0:
                    distPixel = skcolor.deltaE_cie76(i1,colorLAB)
                elif i == 1:
                    distPixel = skcolor.deltaE_cie76(i2,colorLAB)
                elif i == 2:
                    distPixel = skcolor.deltaE_cie76(i3,colorLAB)
                elif i == 3:
                    distPixel = skcolor.deltaE_cie76(i4,colorLAB)
                elif i == 4:
                    distPixel = skcolor.deltaE_cie76(i5,colorLAB)
                else:
                    distPixel = skcolor.deltaE_cie76(i6,colorLAB)
                distPixel = distPixel.flatten()
                #print(distPixel.size)
                for j,threshold in enumerate(Rc):
                    num = sum(1 for t in distPixel  if t <= threshold)/distPixel.size
                    #features.append(num)
                    X[rowNum,colCt] = num
                    
                    colCt+=1
            #print(colCt)
            
            # For features 31-180
            for i in range(6):
                if i == 0:
                    L = i1[:,:,0]
                    a = i1[:,:,1]
                    b = i1[:,:,2]
                elif i == 1:
                    L = i2[:,:,0]
                    a = i2[:,:,1]
                    b = i2[:,:,2]
                elif i == 2:
                    L = i3[:,:,0]
                    a = i3[:,:,1]
                    b = i3[:,:,2]
                elif i == 3:
                    L = i4[:,:,0]
                    a = i4[:,:,1]
                    b = i4[:,:,2]
                elif i == 4:
                    L = i5[:,:,0]
                    a = i5[:,:,1]
                    b = i5[:,:,2]
                else:
                    L = i6[:,:,0]
                    a = i6[:,:,1]
                    b = i6[:,:,2]
                    
                LTemp = L.flatten()
                r = np.sqrt(a**2+b**2).flatten()
                theta = np.degrees(np.arctan2(b,a)).flatten()

                L_diff = np.abs(LTemp - colorLAB[0])
                r_diff = np.abs(r - (np.sqrt(colorLAB[1]**2 + colorLAB[2]**2)))
                theta_diff = np.abs(theta - np.degrees(np.arctan2(colorLAB[2],colorLAB[1])))

                for j,r_th in enumerate(R):
                    l_th = Li[j] # L and R are the same.
                    for theta_th in hueAngle:
                        pixels = 0
                        for p in range(len(L_diff)):
                            if (L_diff[p] <= l_th and r_diff[p] <= r_th and theta_diff[p] <= theta_th) or (L_diff[p] <= l_th and r_diff[p] <= 2.5):
                                pixels += 1
                        X[rowNum,colCt] = pixels/(len(L_diff))
                        colCt+=1
            #print(colCt)
            
            colorCat = category.loc[color,0] 
            X[rowNum,colCt] = categoryImg1[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg2[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg3[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg4[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg5[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg6[colorCat-1]
            colCt+=1
            
            label = data.loc[conceptNum]['col_'+str(color)]
            X[rowNum,colCt] = label
            #print(colCt)
        h = time.time()-t1
        print("\t\t",m, "image: ",h ," seconds")
        t.append(h)
    print(t)
    print("\t** Time elapsed since last category: ",time.time()-t0," seconds **")
strtInd.append(len(X))
print(strtInd)
print("** Total Time elapsed: ",time.time()-init," seconds **")

In [ ]:
strtInd

In [ ]:
## convert your array into a dataframe
xTemp = np.copy(X)
#y = np.copy(X[:,-1])
df = pd.DataFrame(xTemp)
## save to xlsx file

filepath = 'dataCartoon50.xlsx'
df.to_excel(filepath, index=False)
tot = len(xTemp)